In [54]:
import sys
import os
import itertools
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from sklearn.neighbors import NearestNeighbors

import graph as gr
reload(gr)

<module 'graph' from '/home/cstansbu/git_repositories/stx_graph/notebooks/graph.py'>

In [48]:
reload(gr)
fpath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/graph_data/coordinates.csv"
outdir = "/home/cstansbu/temp/graph_data/"

keys = ["ND", "HFD14"]
n = 500 # the number of points to select
metric = 'minkowski'

# load the coordinates
coords = pd.read_csv(fpath)

res = {}

for key in keys:
    k_crd = coords[coords['key'] == key].reset_index()

    # get the node closest to the center
    point = k_crd[['x', 'y']].mean(axis=0)
    nodes = k_crd[['x', 'y']]

    cix = gr.closest_node(point, nodes, metric=metric)
    center  = k_crd.loc[cix, 'nodeId']
    
    # get the n points around the center node
    nbrs = NearestNeighbors(n_neighbors=n,  
                            metric=metric, 
                            algorithm='ball_tree').fit(nodes)
    
    distances, indices = nbrs.kneighbors(nodes)

    neighborhood_idx = indices[cix, :]
    neighborhood = k_crd[k_crd.index.isin(neighborhood_idx)]

    res[key] = neighborhood

    # save the data
    fname = f"{key}_coordinates.pq"
    neighborhood.to_parquet(f"{outdir}{fname}", index=False)

print('done')

done


In [50]:
dirpath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/graph_data/edge_lists/"

outdir = "~/temp/"

col = ['node1', 'node2', 'key', 'M5_M5']

for f in os.listdir(dirpath):
    if 'harmonic' in f:
        key = f.split("_")[0]
        if key in keys:
            tmp = pd.read_csv(f"{dirpath}{f}", usecols=col)

            # get the nieghborhood from above
            nieghborhood = res[key]['nodeId'].to_list()
            tmp = tmp[(tmp['node1'].isin(nieghborhood)) & (tmp['node2'].isin(nieghborhood))]
        
            tmp.to_parquet(f"{outdir}{key}_edges.csv", index=False)        

print('done')

done
